In [3]:
import yfinance as yf
import pandas as pd

def compute_ddm_with_sector_growth(ticker_symbol, sector_growth_dict=None, risk_free_rate=0.042, market_return=0.09):
    try:
        ticker = yf.Ticker(ticker_symbol)
        info = ticker.info

        # Get current dividend per share
        d0 = info.get('dividendRate', None)
        if d0 is None or d0 == 0:
            return {'Ticker': ticker_symbol, 'Error': 'No dividend'}

        # Get beta and compute cost of equity via CAPM
        beta = info.get('beta', None)
        if beta is None:
            return {'Ticker': ticker_symbol, 'Error': 'No beta'}

        cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)

        # Get sector and assign sector-specific growth rate
        sector = info.get('sector', 'Unknown')
        if sector_growth_dict is None:
            sector_growth_dict = {
                'Technology': 0.07,
                'Consumer Defensive': 0.04,
                'Consumer Cyclical': 0.06,
                'Healthcare': 0.05,
                'Industrials': 0.04,
                'Financial Services': 0.05,
                'Energy': 0.03,
                'Utilities': 0.03,
                'Basic Materials': 0.04,
                'Real Estate': 0.03,
                'Communication Services': 0.05,
                'Unknown': 0.04
            }

        g = sector_growth_dict.get(sector, 0.04)

        # Safety check: r > g
        if cost_of_equity <= g:
            return {'Ticker': ticker_symbol, 'Error': f"r <= g (r={cost_of_equity:.2f}, g={g:.2f})"}

        # DDM Formula (per share)
        d1 = d0 * (1 + g)
        value_per_share = d1 / (cost_of_equity - g)

        # Multiply by shares outstanding to get firm value
        shares_outstanding = info.get('sharesOutstanding', None)
        if shares_outstanding is None:
            return {'Ticker': ticker_symbol, 'Error': 'No shares outstanding'}

        firm_value = value_per_share * shares_outstanding
        firm_value_billions = firm_value / 1e9  # convert to billions

        return {
            'Ticker': ticker_symbol,
            'Sector': sector,
            'Dividend (D0)': round(d0, 2),
            'Growth Rate': round(g, 4),
            'Cost of Equity': round(cost_of_equity, 4),
            'Value per Share': round(value_per_share, 2),
            'Shares Outstanding': round(shares_outstanding / 1e9, 2),  # in billions
            'Intrinsic Firm Value (B)': round(firm_value_billions, 2)
        }

    except Exception as e:
        return {'Ticker': ticker_symbol, 'Error': str(e)}


TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', "PG", "MMM"]  # Replace with your 1000 tickers

ddm_results = [compute_ddm_with_sector_growth(t) for t in TICKERS]
pd.DataFrame(ddm_results)

,Ticker,Sector,Dividend (D0),Growth Rate,Cost of Equity,Value per Share,Shares Outstanding,Intrinsic Firm Value (B),Error
0,AAPL,Technology,1.04,0.07,0.0979,39.86,14.84,591.49,NaN
1,MSFT,Technology,3.32,0.07,0.0926,156.91,7.43,1166.32,NaN
2,GOOGL,Communication Services,0.84,0.05,0.0907,21.69,5.82,126.15,NaN
3,AMZN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No dividend
4,META,Communication Services,2.10,0.05,0.1031,41.52,2.17,90.06,NaN
5,PG,Consumer Defensive,4.23,0.04,0.0599,221.02,2.34,517.71,NaN
6,MMM,Industrials,2.92,0.04,0.0950,55.22,0.53,29.41,NaN


In [ ]:
# FOCUS ONLY ON STEADY DIVIDEND PAYING FIRMS
# FOR SURE EXCLUDE TECHNOLOGY COMPANIES